# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.2.0-bin-hadoop2.7')

from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [3]:
df = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
df.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [7]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager because its easy enough, but keep in mind it probably won't be any sort of a signal because the agency mentioned its randomly assigned!

In [9]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [10]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [11]:
output = assembler.transform(df)

In [12]:
final_data = output.select('features','churn')

### Build the model

In [13]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [14]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol='churn')

In [15]:
fitted_lr = lr.fit(train_churn)

In [16]:
training_sum = fitted_lr.summary

In [17]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                637|                637|
|   mean|0.17111459968602827|0.12872841444270017|
| stddev|0.37690503248554424|0.33516228589493546|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
pred_and_labels = fitted_lr.evaluate(test_churn)

In [20]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.59289496628588...|[0.64402910935476...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.46143498213436...|[0.99577050823600...|       0.0|
|[28.0,8670.98,0.0...|    0|[7.72345882884485...|[0.99955786754952...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.70685182772134...|[0.93742974598476...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.73790205519353...|[0.99131902069651...|       0.0|
|[29.0,11274.46,1....|    0|[4.46325271765637...|[0.98860649278223...|       0.0|
|[29.0,12711.15,0....|    0|[5.43268907577592...|[0.99564770015870...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.27722021583748...|[0.98630885423587...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.16549690318491...|[0.95951501902889...|       0.0|
|[30.0,10960.52,

### AUC

In [21]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [22]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [23]:
auc

0.7769171610635025

### Accuracy

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
# For multiclass
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='churn',
                                             metricName='accuracy')

In [26]:
acc = evaluator.evaluate(pred_and_labels.predictions)

In [27]:
acc

0.908745247148289

### Predict on brand new unlabeled data

We still need to evaluate the new_customers.csv file!

In [29]:
# Use the whole original data to fit the model
final_lr_model = lr.fit(final_data)

In [30]:
# Read new customer data
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [31]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [32]:
# Apply the same vector assembler
test_new_customers = assembler.transform(new_customers)

In [35]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [39]:
final_results = final_lr_model.transform(test_new_customers)

In [40]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!